In [3]:
import pydub
import numpy as np
import os
import sounddevice as sd
from pesq import pesq
from copy import deepcopy
import librosa

In [4]:
audios=[]
def find_(name:str):
    result=[]
    for idx,ch in enumerate(name):
        if ch=="_":
            result.append(idx)
    return result
for dirpath, dirnames, filenames in os.walk("./audio1"):
    for file_name in filenames:
        if file_name==".DS_Store":
            continue
        audio_data={}
        audio_data["format"]=file_name[-3:]
        i_=find_(file_name)
        audio_data["type"]=file_name[:i_[0]]
        audio_data["bits"]=file_name[i_[-1]+1:-4]
        full_path=os.path.join(dirpath, file_name)
        dub_temp=pydub.AudioSegment.from_file(full_path, format=audio_data["format"]) 
        samples=dub_temp.get_array_of_samples()
        sample_rate=dub_temp.frame_rate
        audio_data["freq"]=sample_rate
        audio_data["data"]=samples
        audio_data["size"]=os.path.getsize(full_path)
        audios.append(audio_data)

In [5]:
# sd.play(audios[0]["data"], audios[0]["freq"])


In [6]:
for audio in audios:
    if audio["type"]=="原始音乐":
        music_origin=audio
    elif audio["type"]=="原始语音":
        speech_origin=audio

In [7]:
# sd.play(music_origin["data"], music_origin["freq"] )
np.min(np.array(music_origin["data"]))

np.int32(-1932735233)

In [ ]:
def compare_with_mel(ref:dict, deg:dict, n_mels=128, frame_len=0.025, hop_len=0.01, eps=1e-10):
    ref_data=np.array(ref["data"], dtype=np.float64)
    deg_data=np.array(deg["data"], dtype=np.float64)
    ref_data=ref_data/np.max(np.abs(ref_data))
    deg_data=deg_data/np.max(np.abs(deg_data))
    # print(ref_data)
    # print(deg)
    if ref["freq"]!=deg["freq"]:
        deg_data=librosa.resample(deg_data, orig_sr=deg["freq"], target_sr=ref["freq"])
    min_len=min(len(ref["data"]), len(deg["data"]))
    
    ref_data=ref_data[:min_len]
    deg_data=deg_data[:min_len]
    n_fft=int(frame_len*ref["freq"])
    hop=int(hop_len*ref["freq"])
    S_orig=librosa.feature.melspectrogram(y=deg_data, sr=ref["freq"], n_mels=n_mels, n_fft=n_fft, hop_length=hop)
    S_deg=librosa.feature.melspectrogram(y=deg_data, sr=deg["freq"], n_mels=n_mels, n_fft=n_fft, hop_length=hop)
    los_S_orig=np.log(S_orig+eps)
    log_S_deg=np.log(S_deg+eps)
    P_signal=np.mean(los_S_orig**2)
    D=los_S_orig-log_S_deg
    P_noise=np.mean(D**2)
    if P_noise==0:
        mel_snr=float('inf')
    else:
        mel_snr=10*np.log10(P_signal/P_noise)
    return mel_snr

In [9]:
for idx,audio in enumerate(audios):
    if audio["type"]=="音乐":
        score=compare_with_mel(ref=music_origin, deg=audio)
        audios[idx]["score"]=score 
    elif audio["type"]=="语音":
        score=compare_with_mel(ref=speech_origin, deg=audio)
        audios[idx]["score"]=score 
    else:
        audios[idx]["score"]=float('inf')

In [10]:
for idx,audio in enumerate(audios):
    print(f"idx:{idx}, Type:{audio["type"]}, format:{audio["format"]}, freq:{audio["freq"]}, score:{audio["score"]}, ")

idx:0, Type:语音, format:wav, freq:16000, score:18.314466718611957, 
idx:1, Type:音乐, format:mp3, freq:44100, score:18.524451023312423, 
idx:2, Type:音乐, format:wav, freq:48000, score:inf, 
idx:3, Type:音乐, format:mp3, freq:44100, score:18.426748281117302, 
idx:4, Type:语音, format:wav, freq:44100, score:27.050768329974176, 
idx:5, Type:语音, format:mp3, freq:44100, score:27.03502176179284, 
idx:6, Type:语音, format:mp3, freq:16000, score:20.066914738866966, 
idx:7, Type:音乐, format:wav, freq:8000, score:7.467198641688583, 
idx:8, Type:音乐, format:wav, freq:32000, score:11.67471811955007, 
idx:9, Type:语音, format:wav, freq:16000, score:20.37355769216617, 
idx:10, Type:语音, format:wav, freq:22050, score:18.685708516542235, 
idx:11, Type:音乐, format:wav, freq:48000, score:inf, 
idx:12, Type:原始音乐, format:wav, freq:48000, score:inf, 
idx:13, Type:语音, format:wav, freq:8000, score:inf, 
idx:14, Type:语音, format:mp3, freq:44100, score:26.964155764511744, 
idx:15, Type:语音, format:wav, freq:44100, score:28.0897

In [17]:
summary_mp3=0
summary_acc=0
summary_wav=0
for audio in audios:
    if audio["freq"]==44100:
        if audio["format"]=="mp3":
           summary_mp3+=audio["score"]
        elif audio["format"]=="m4a":
            summary_acc+=audio["score"]
        elif audio["format"]=="wav":
            summary_wav+=audio["score"]
print(f"sc_mp3:{summary_mp3}, sc_acc:{summary_acc}, sc_wav:{summary_wav}")

sc_mp3:181.41308607370675, sc_acc:135.6769137146322, sc_wav:87.36340768897529


In [19]:
sizes=[audio["size"] for audio in audios]
print(sizes)

[95525, 402328, 960044, 80500, 526382, 241414, 48716, 160044, 640044, 191006, 263214, 1440044, 1440044, 95526, 96591, 263213, 55939, 572928, 160957, 441044, 882044, 190683, 33980, 48317, 161900, 441044, 43845, 190829, 144865, 320044, 381968, 859370, 97388, 167226, 241414, 859370]


In [ ]:
w_size=0.3
w_loss=0.4
w_compute=0.15
w_suit=0.15 
suit_scores={}

In [ ]:
import numpy as np

def entropy_weight_method(data):
    """
    熵权法计算指标权重
    参数:
        data: numpy 数组，行表示样本，列表示指标
    返回:
        weights: 各指标的权重
    """
    # 数据预处理：确保数据为正值（熵权法要求数据非负）
    data = np.array(data, dtype=float)
    if (data < 0).any():
        raise ValueError("数据中包含负值，请确保所有数据非负！")

    # 1. 数据标准化（归一化到 [0,1]）
    # 使用线性归一化：(x - min) / (max - min)
    min_val = np.min(data, axis=0)
    max_val = np.max(data, axis=0)
    # 防止除以零
    denominator = max_val - min_val
    denominator[denominator == 0] = 1  # 如果某列最大值等于最小值，设分母为 1
    normalized_data = (data - min_val) / denominator

    # 2. 计算比重 p_ij
    # 防止除以零，先对归一化数据加一个小值（如 1e-10）
    normalized_data += 1e-10
    row_sums = np.sum(normalized_data, axis=1, keepdims=True)
    p = normalized_data / row_sums

    # 3. 计算熵值 e_j
    m = data.shape[0]  # 样本数
    k = 1 / np.log(m)  # 常数 k = 1/ln(m)
    entropy = -k * np.sum(p * np.log(p), axis=0)

    # 4. 计算权重 w_j
    d = 1 - entropy  # 差异系数
    weights = d / np.sum(d)  # 归一化权重

    return weights

# 示例数据
if __name__ == "__main__":
    # 假设有 4 个样本，3 个指标
    data = np.array([
        [5, 2, 9],
        [3, 4, 7],
        [4, 3, 8],
        [6, 5, 6]
    ])
     
    try:
        weights = entropy_weight_method(data)
        print("各指标权重：", weights)
    except ValueError as e:
        print("错误：", e)

各指标权重： [0.29255174 0.37872025 0.32872801]
